In [1]:
import sys
import os
import json

from pprint import pprint


nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

    
project_dir = os.path.join(os.getcwd(), '..')
print(sys.version)
print(os.getcwd())

pprint(sys.path)


3.6.5 (default, May  3 2018, 10:08:28) 
[GCC 5.4.0 20160609]
/home/viktor/dev/recept-scanning/notebooks
['',
 '/home/viktor/venvs/receipt/lib/python36.zip',
 '/home/viktor/venvs/receipt/lib/python3.6',
 '/home/viktor/venvs/receipt/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '/home/viktor/venvs/receipt/lib/python3.6/site-packages',
 '/home/viktor/venvs/receipt/lib/python3.6/site-packages/IPython/extensions',
 '/home/viktor/.ipython',
 '/home/viktor/dev/recept-scanning']


In [2]:
import json
from recread.receipt.core import read_receipt_from_google_ocr_json

RECEIPT_FILE_NAME_KEISER = 'kvittering-keiser.jpg.json'
RECEIPT_FILE_NAME_KIWI = 'kvittering-kiwi.jpg.json'
RECEIPT_FILE_NAME_IKEA = 'kvittering-ikea.jpg.json'
RECEIPT_FILE_NAME_HOLDBART = 'receipt_holdbart.json'

with open(os.path.join(project_dir, 'data', 'receipt_responses', RECEIPT_FILE_NAME_KEISER)) as json_file:
    json_ocr_keiser = json.load(json_file)
with open(os.path.join(project_dir, 'data', 'receipt_responses', RECEIPT_FILE_NAME_KIWI)) as json_file:
    json_ocr_kiwi = json.load(json_file)
with open(os.path.join(project_dir, 'data', 'receipt_responses', RECEIPT_FILE_NAME_IKEA)) as json_file:
    json_ocr_ikea = json.load(json_file)
with open(os.path.join(project_dir, 'data', 'receipt_responses', RECEIPT_FILE_NAME_HOLDBART)) as json_file:
    json_ocr_holdbart = json.load(json_file)
    
receipt_keiser = read_receipt_from_google_ocr_json(json_ocr_keiser)
receipt_kiwi = read_receipt_from_google_ocr_json(json_ocr_kiwi)
receipt_ikea = read_receipt_from_google_ocr_json(json_ocr_ikea)
receipt_holdbart = read_receipt_from_google_ocr_json(json_ocr_holdbart)


READING RECEIPT!!!
[(1, 54), (3, 29), (5, 29)]
READING RECEIPT!!!
[(1, 86), (3, 51), (5, 51)]
READING RECEIPT!!!
[(1, 114), (3, 54), (5, 39), (7, 61)]
READING RECEIPT!!!
[(1, 213), (3, 114), (5, 81), (7, 74), (9, 134)]


In [29]:
import re
import random

compiled_number_pattern = re.compile(r'\d')

def preprocess_string(string):
    return compiled_number_pattern.sub('#', string.lower())

def print_predicted_labels(predicted, strings):
    for p, s in zip(predicted, strings):
        print(p, s)
        
def write_train_test_files(lines, test_size=.2):
    with open(os.path.join(project_dir, 'ft', 'receipt_lines_labeled_tokens.train.txt'), 'w') as train_file:
        with open(os.path.join(project_dir, 'ft', 'receipt_lines_labeled_tokens.test.txt'), 'w') as test_file:
            for line in lines:
                if random.random() < test_size:
                    test_file.write(line)
                else: 
                    train_file.write(line)
                    
    

In [30]:
import json

with open(os.path.join(project_dir, 'ft', 'receipt_lines_labeled_tokens_corrected.txt'), 'r') as in_file:
    write_train_test_files(in_file.readlines(), test_size=.4)
    

In [4]:
from recread.parsing.lines import get_line_type
from recread.parsing.enums import line_types


with open(os.path.join(project_dir, 'ft', 'receipt_lines_labeled_tokens.txt'), 'w') as outfile:
    for line_type, line in [(x.type, ' '.join(x.token_line)) for x in receipt_holdbart.receipt_lines + receipt_ikea.receipt_lines + receipt_keiser.receipt_lines + receipt_kiwi.receipt_lines]:
        outfile.write(f'__label__{True if line_type == line_types.PRODUCT else False} {preprocess_string(line)}{os.linesep}')
outfile.close()

In [24]:
import fasttext

classifier = fasttext.supervised(
    os.path.join(project_dir, 'ft', 'receipt_lines_labeled_tokens_corrected.txt'), 
    os.path.join(project_dir, 'ft', 'model'), 
    epoch=100,
    bucket=100000,
    word_ngrams=3,
    maxn=5,
)


In [25]:
strings = [x.lower() for x in [
    'Lime16,87', 
    'Huriye', 
    '0,270kgxkr83,00', 
    'VillBlåbærM/MørkSjokolade114,90', 
    'VillBlåbærM/MørkSjokolade14,90', 
    '1TinemelkhelIT17,90',
    '0,338kgX49,90/kg',
    'anus',
    'penis101,11',
]]

token_strings = [preprocess_string(x) for x in [
    'Lime 16 , 87', 
    'Huriye', 
    '0 , 270 kg x kr 83 , 00', 
    'Vill Blåbær M / Mørk Sjokolade 1 14 , 90', 
    'Vill Blåbær M / Mørk Sjokolade 14 , 90', 
    '1 Tinemelkhel IT 17 , 90',
    '0 , 338 kg X 49 , 90 / kg',
    'anus',
    'CIABATTA MINI 4x65G ELD 15 % 12 , 00',
    'penis 101 , 11',
]]

predicted_token_labels = classifier.predict_proba(token_strings)


print_predicted_labels(predicted_token_labels, token_strings)

[('True', 0.974609)] lime ## , ##
[('False', 0.998047)] huriye
[('False', 0.662109)] # , ### kg x kr ## , ##
[('True', 0.970703)] vill blåbær m / mørk sjokolade # ## , ##
[('True', 0.921875)] vill blåbær m / mørk sjokolade ## , ##
[('True', 0.966797)] # tinemelkhel it ## , ##
[('False', 0.990234)] # , ### kg x ## , ## / kg
[('False', 1.0)] anus
[('True', 0.626953)] ciabatta mini #x##g eld ## % ## , ##
[('False', 0.820312)] penis ### , ##


In [50]:
trained_classifier = fasttext.supervised(
    os.path.join(project_dir, 'ft', 'receipt_lines_labeled_tokens.train.txt'), 
    os.path.join(project_dir, 'ft', 'train_model'), 
    epoch=50,
    bucket=1000,
    word_ngrams=3,
    lr=.5,
)

result = trained_classifier.test(os.path.join(project_dir, 'ft', 'receipt_lines_labeled_tokens.test.txt'))
pprint(result.__dict__)

{'nexamples': 68, 'precision': 0.9558823529411765, 'recall': 0.9558823529411765}


In [52]:
pprint(receipt_kiwi.get_json_dict())

{'products': [{'items_quantity': None,
               'name': 'VANNM/KULLSYREF.PRICE',
               'price': 4.9,
               'quantity': None,
               'unit_price': None},
              {'items_quantity': None,
               'name': '+Pant',
               'price': 2.5,
               'quantity': None,
               'unit_price': None},
              {'items_quantity': None,
               'name': 'VANNM/KULLSYRELIME',
               'price': 4.9,
               'quantity': None,
               'unit_price': None},
              {'items_quantity': None,
               'name': '+Pant',
               'price': 2.5,
               'quantity': None,
               'unit_price': None},
              {'items_quantity': None,
               'name': 'FRYDENLUNDPILSNER',
               'price': 28.9,
               'quantity': {'unit': 'L', 'value': 0.5},
               'unit_price': None},
              {'items_quantity': None,
               'name': '+Pant',
               'pri